Este código es parte de un proyecto destinado a utilizar una red neuronal para clasificar música por género a través de espectrogramas. Las bibliotecas importadas permiten manejar archivos de audio (os), realizar operaciones numéricas (numpy), procesar y visualizar audio (librosa), construir y entrenar modelos de redes neuronales (tensorflow), y dividir datos para entrenamiento y prueba (train_test_split). En resumen, este script está configurado para tomar archivos de audio, transformarlos en espectrogramas, y luego alimentar esos datos a una red neuronal para entrenarla a reconocer y clasificar diferentes géneros musicales. 🎧🎶

In [ ]:
import os
import numpy as np
import librosa
import librosa.display
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

La función extract_spectrogram se encarga de tomar la ruta de un archivo de audio y generar un espectrograma mel, que es una representación visual de las frecuencias en el audio a lo largo del tiempo.


* Primero, la función carga el archivo de audio usando librosa.load.
* Luego, crea un espectrograma mel mediante librosa.feature.melspectrogram, definiendo el número de bandas mel (n_mels) y la frecuencia máxima (fmax).
*   Finalmente, convierte el espectrograma a una escala logarítmica con librosa.power_to_db para que sea más interpretable y retorna este espectrograma logarítmico.





In [ ]:
def extract_spectrogram(file_path, n_mels=128, fmax=8000):
    y, sr = librosa.load(file_path, sr=None)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, fmax=fmax)
    log_S = librosa.power_to_db(S, ref=np.max)
    return log_S


La función load_gtzan_data está diseñada para cargar datos del conjunto de datos GTZAN, que contiene música clasificada por géneros. Toma la ruta al directorio GTZAN, una lista de géneros y un número opcional de bandas mel (n_mels). La función recorre cada género y cada archivo de audio, extrayendo su espectrograma con la función extract_spectrogram. Si el espectrograma no tiene el tamaño adecuado, se redimensiona o se rellena. Finalmente, los espectrogramas se almacenan en la lista X y las etiquetas (índices de los géneros) en la lista y. Ambas listas se convierten en arrays de NumPy y se retornan.

In [ ]:
def load_gtzan_data(gtzan_path, genres, n_mels=128):
    X = []
    y = []
    for genre in genres:
        genre_dir = os.path.join(gtzan_path, genre)
        for file_name in os.listdir(genre_dir):
            file_path = os.path.join(genre_dir, file_name)
            spectrogram = extract_spectrogram(file_path, n_mels=n_mels)
            if spectrogram.shape[1] > 128:
                spectrogram = spectrogram[:, :128]
            elif spectrogram.shape[1] < 128:
                spectrogram = np.pad(spectrogram, ((0, 0), (0, 128 - spectrogram.shape[1])), mode='constant')

            X.append(spectrogram)
            y.append(genres.index(genre))
    return np.array(X), np.array(y)

In [ ]:
GTZAN_PATH = '/content/drive/MyDrive/Tesis/GZTAN/genres_original'
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

In [ ]:
X, y = load_gtzan_data(GTZAN_PATH, genres)

In [ ]:
X = X[..., np.newaxis]
X = X / np.max(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = models.Sequential()

In [ ]:
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.0926 - loss: 3560219.7500 - val_accuracy: 0.1100 - val_loss: 2.3024
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.0969 - loss: 2.3026 - val_accuracy: 0.0650 - val_loss: 2.3029
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.0859 - loss: 2.3027 - val_accuracy: 0.0650 - val_loss: 2.3032
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.0871 - loss: 2.3026 - val_accuracy: 0.0650 - val_loss: 2.3036
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.1009 - loss: 2.3022 - val_accuracy: 0.0650 - val_loss: 2.3042
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.0875 - loss: 2.3024 - val_accuracy: 0.0650 - val_loss: 2.3046
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.1011 - loss: 2.3022 - val_accuracy: 0.0650 - val_loss: 2.3053
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.1147 - loss: 2.3018 - val_accuracy: 0.0650 - val

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Precisión en el conjunto de prueba: {test_acc:.2f}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 211ms/step - accuracy: 0.0729 - loss: 2.3129
Precisión en el conjunto de prueba: 0.06
